In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


##  Job listings data

In [9]:
job_listings = pd.DataFrame({
    'job_id': [1, 2, 3],
    'title': ['Data Scientist', 'Software Engineer', 'Product Manager'],
    'description': ['Seeking a data scientist with experience in machine learning.',
                    'Looking for a software engineer proficient in Python and Java.',
                    'Product manager with strong communication and leadership skills.']
})


## User profiles data

In [11]:
user_profiles = pd.DataFrame({
    'user_id': [1, 2],
    'skills': ['Python, Machine Learning', 'Java, Communication']})

## Collaborative Filtering

In [12]:
def calculate_user_similarity(user1_skills, user2_skills):
    user1_skills = set(user1_skills.split(', '))
    user2_skills = set(user2_skills.split(', '))
    intersection = len(user1_skills.intersection(user2_skills))
    union = len(user1_skills.union(user2_skills))
    return intersection / union

def recommend_jobs_collaborative(user_profile, job_listings, num_recommendations=3):
    job_listings['similarity'] = job_listings['description'].apply(
        lambda x: calculate_user_similarity(user_profile['skills'], x))
    recommended_jobs = job_listings.sort_values(by='similarity', ascending=False).head(num_recommendations)
    return recommended_jobs[['job_id', 'title', 'description']]


## Content-Based Filtering

In [13]:
def recommend_jobs_content_based(user_profile, job_listings, num_recommendations=3):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(job_listings['description'])
    user_profile_vector = tfidf_vectorizer.transform([user_profile['skills']])
    similarities = cosine_similarity(user_profile_vector, tfidf_matrix).flatten()
    similar_jobs_indices = similarities.argsort()[-num_recommendations:][::-1]
    recommended_jobs = job_listings.iloc[similar_jobs_indices][['job_id', 'title', 'description']]
    return recommended_jobs

In [14]:
user_profile = user_profiles.iloc[0]

## Collaborative Filtering recommendations

In [15]:
collab_recommendations = recommend_jobs_collaborative(user_profile, job_listings)
print("Collaborative Filtering Recommendations:")
print(collab_recommendations)

Collaborative Filtering Recommendations:
   job_id              title  \
0       1     Data Scientist   
1       2  Software Engineer   
2       3    Product Manager   

                                         description  
0  Seeking a data scientist with experience in ma...  
1  Looking for a software engineer proficient in ...  
2  Product manager with strong communication and ...  


## Content-Based Filtering recommendations

In [16]:
content_based_recommendations = recommend_jobs_content_based(user_profile, job_listings)
print("\nContent-Based Filtering Recommendations:")
print(content_based_recommendations)


Content-Based Filtering Recommendations:
   job_id              title  \
0       1     Data Scientist   
1       2  Software Engineer   
2       3    Product Manager   

                                         description  
0  Seeking a data scientist with experience in ma...  
1  Looking for a software engineer proficient in ...  
2  Product manager with strong communication and ...  
